# Optimal Control
This section provides a brief introduction to the package's interface for sequence optimization. We use the Cramer-Rao bound](https://en.wikipedia.org/wiki/Cramér–Rao_bound) (CRB) to assess a sequence's performance and optimize the amplitudes (ω1) and durations (TRF) of a train of RF-pulses to reduce the CRB, assuming a [Balanced Hybrid-State Free Precession Pulse Sequence. For computational efficiency, the derivatives of the CRB wrt. ω1 and TRF are calcuated with the adjoint state method as known from the [optimal control literature](https://www.sciencedirect.com/science/article/pii/S1090780703001538).

For this tutorial, we use the following packages:

In [ ]:
using MRIgeneralizedBloch
using LinearAlgebra
BLAS.set_num_threads(1) # single threaded is faster in this case
using Optim             # provides the optimization algorithm
using Plots

Here, we optimize the pulse sequence for a predefined set of parameters:

In [ ]:
m0s = 0.15
R1f = 0.5   # 1/s
R2f = 15    # 1/s
Rx = 30     # 1/s
R1s = 3     # 1/s
T2s = 10e-6 # s
ω0 = 0      # rad/s
B1 = 1;     # in units of B1_nominal

and we optimize

In [ ]:
Npulse = 200;

pulses, spaced

In [ ]:
TR = 3.5e-3; # s

apart. A cyle duration of

In [ ]:
Npulse * TR

seconds is much shorter than the optimal durations, which are roughly in the range of 4-10s, but we reduce `Npulse` speed up the computations in this demonstration. We precompute the linear the Linear Approximation of the generalized Bloch model:

In [ ]:
R2slT = precompute_R2sl();

In the calculation of the CRB, we account for following gradients:

In [ ]:
grad_list = [grad_m0s(), grad_R1f(), grad_R2f(), grad_Rx(), grad_R1s(), grad_T2s(), grad_ω0(), grad_B1()];

and we sum the CRB of all parameters, weighted by the following vector:

In [ ]:
weights = transpose([0, 1, 0, 0, 0, 0, 0, 0, 0]);

Note that the vector `weights` has one more entry compared to the `grad_list` vector, as the first value always indicates the derivative wrt. $M_0$, which is calculated in any case. Here, we only optimize for the CRB of $m_0^s$, while accounting for a fit of all 9 model parameters. Hence, we can use an arbitrary weighting for the CRB of $m_0^s$.

We take some initial guess for the pulse train:

In [ ]:
α = abs.(sin.((1:Npulse) * 2π/Npulse));

initialize with a constant `TRF = 300μs`:

In [ ]:
TRF = 300e-6 .* one.(α);

and define that first RF pulse as a 500μs inversion pulse by modifying vectors accordingly and by creating a bit vector that indicates the position of the inversion pulse:

In [ ]:
α[1] = π # first pulse is an inversion pulse
TRF[1] = 500e-6 # first pulse is an inversion pulse
isInversionPulse = [true, falses(length(α)-1)...];

We calculate the initial `ω1`

In [ ]:
ω1 = α ./ TRF;

and plot the inital control:

In [ ]:
p1 = plot(TR*(1:Npulse), α ./ π, ylabel="α/π")
p2 = plot(TR*(1:Npulse), 1e6TRF, ylim=(0, 1e3), xlabel="t (s)", ylabel="TRF (μs)")
p = plot(p1, p2, layout=(2, 1), legend=:none)

With above defined weights, the function CRB_gradient_OCT returns the CRB

In [ ]:
(CRBm0s, grad_ω1, grad_TRF) = MRIgeneralizedBloch.CRB_gradient_OCT(ω1, TRF, TR, ω0, B1, m0s, R1f, R2f, Rx, R1s, T2s, R2slT, grad_list, weights, isInversionPulse=isInversionPulse)
CRBm0s

along with the gradients:

In [ ]:
p1 = plot(TR*(1:Npulse), grad_ω1  .* ((-1) .^ (1:Npulse)), ylabel="∂CRB(m0s) / ∂ω1 (s/rad)")
p2 = plot(TR*(1:Npulse), grad_TRF .* ((-1) .^ (1:Npulse)), ylabel="∂CRB(m0s) / ∂TRF (1/s)", xlabel="t (s)")
p = plot(p1, p2, layout=(2, 1), legend=:none)

Note that we remove the oszillating nature of the gradient here for the display.

In this example, we limit the control to the following bounds

In [ ]:
ω1_min  = 0      # rad/s
ω1_max  = 2e3π   # rad/s
TRF_min = 100e-6 # s
TRF_max = 500e-6; # s

and the function `bound_ω1_TRF!` modifies `ω1` and `TRF` to comply with these bounds and returns a single vector in the range `[-Inf, Inf]` that relates to the bounded control by a `tanh` transformation:

In [ ]:
x0 = MRIgeneralizedBloch.bound_ω1_TRF!(ω1, TRF; ω1_min = ω1_min, ω1_max = ω1_max, TRF_min = TRF_min, TRF_max = TRF_max)

Further, we initialze a gradient of the same length:

In [ ]:
G = similar(x0);

and define a cost function:

In [ ]:
function fg!(F, G, x)
    ω1, TRF = MRIgeneralizedBloch.get_bounded_ω1_TRF(x; ω1_min = ω1_min, ω1_max = ω1_max, TRF_min = TRF_min, TRF_max = TRF_max)

    (F, grad_ω1, grad_TRF) = MRIgeneralizedBloch.CRB_gradient_OCT(ω1, TRF, TR, ω0, B1, m0s, R1f, R2f, Rx, R1s, T2s, R2slT, grad_list, weights, isInversionPulse=isInversionPulse)
    F = abs(F)

    F += MRIgeneralizedBloch.second_order_α!(grad_ω1, grad_TRF, ω1, TRF; λ=1e4)
    F += MRIgeneralizedBloch.RF_power!(grad_ω1, grad_TRF, ω1, TRF, λ=1e-3, Pmax=3e6, TR=TR)
    F += MRIgeneralizedBloch.TRF_TV!(grad_TRF, ω1, TRF; λ=1e3)

    MRIgeneralizedBloch.apply_bounds_to_grad!(G, x, grad_ω1, grad_TRF; ω1_min = ω1_min, ω1_max = ω1_max, TRF_min = TRF_min, TRF_max = TRF_max)
    return F
end;

We use the packge Optim.jl](https://julianlsolvers.github.io/Optim.jl/stable/), which requires the cost function `fg!(F, G, x)` to take the cost, the gradient, and the control as input variables and it will over-writes the gradient in place. The cost function calculates the gradient of the CRB with above described optimal control code and we, further, add some regularization terms: [`second_order_α!` penalizes the curvature of α, which results in smoother flip angle trains and helps ensuring the hybrid state conditions](https://www.nature.com/articles/s42005-019-0174-0). The penalty [`RF_power!` penalizes the power deposition of the RF-pulse train if $sum(ω_1^2 ⋅ T_\text{RF}) / T_\text{cycle} ≥ Pmax$ and helps with compliance with safety limits. Assuming a reasonble `λ`, the resulting enery will be `Pmax` in units of (rad/s)² and averaged over the cycle. The value `Pmax=3e6` (rad/s)² heuristically proofed to run reliably on a 3T system. The penalty `TRF_TV!` penalizes fast fluctuations of $T_\text{RF}$, as know that fluctuations of $ω_1$ and $T_\text{RF}$ have negligible effect if they are fast than the biophysical time constants. We note, however, that this penalty is not required and rather ensure *beauty* of the result.

With all this in place, we can start the actual optimization

In [ ]:
result = optimize(Optim.only_fg!(fg!), # cost function
    x0,                                # initialization
    BFGS(),                            # algorithm
    Optim.Options(show_trace=true,     # show cost function
        iterations=10_000,             # larger number as we use a time limit
        time_limit=(15*60)             # in seconds
        )
    )

and see that the CRB(m0s) has reduced substantially:

In [ ]:
(CRBm0s, grad_ω1, grad_TRF) = MRIgeneralizedBloch.CRB_gradient_OCT(ω1, TRF, TR, ω0, B1, m0s, R1f, R2f, Rx, R1s, T2s, R2slT, grad_list, weights, isInversionPulse=isInversionPulse)
CRBm0s

After transfering the result back into the space of bounded $ω_1$ and $T_\text{RF}$ values

In [ ]:
ω1, TRF = MRIgeneralizedBloch.get_bounded_ω1_TRF(result.minimizer; ω1_min = ω1_min, ω1_max = ω1_max, TRF_min = TRF_min, TRF_max = TRF_max);

and calculating the flip angle, we can plot the optimized control:

In [ ]:
α = ω1 .* TRF
p1 = plot(TR*(1:Npulse), α ./ π, ylabel="α/π")
p2 = plot(TR*(1:Npulse), 1e6TRF, ylim=(0, 1e3), xlabel="t (s)", ylabel="TRF (μs)")
p = plot(p1, p2, layout=(2, 1), legend=:none)

To analyze the results, we can calculate and plot all magnetization components:

In [ ]:
m = calculatesignal_linearapprox(α, TRF, TR, ω0, B1, m0s, R1f, R2f, Rx, R1s, T2s, R2slT; output=:realmagnetization)
m = vec(m)

xf = [m[i][1] for i=1:length(m)]
yf = [m[i][2] for i=1:length(m)]
zf = [m[i][3] for i=1:length(m)]
xs = [m[i][4] for i=1:length(m)]
zs = [m[i][5] for i=1:length(m)]

p = plot(xlabel="t (s)", ylabel="m (normalized)")
plot!(p, TR*(1:Npulse), xf ./(1-m0s), label="xᶠ")
plot!(p, TR*(1:Npulse), yf ./(1-m0s), label="yᶠ")
plot!(p, TR*(1:Npulse), zf ./(1-m0s), label="zᶠ")
plot!(p, TR*(1:Npulse), xs ./   m0s , label="xˢ")
plot!(p, TR*(1:Npulse), zs ./   m0s , label="zˢ")

And we can also plot the dynamics of the free spin pool on the Bloch sphere:

In [ ]:
p = plot(xf, zf, xlabel="xf", ylabel="zf")

As yᶠ is close to zero in this particular case, we neglect it in this 2D plot.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*